In [1]:
import json
import requests
import csv
import configparser
import warnings
import uuid
import pandas as pd
import sqlite3 as sl
import re





In [2]:
my_data  = pd.read_csv('my_data.secret', sep=',',index_col='parameters')
my_data.astype(str)

devUser=my_data.loc['devUser'].values [0]
devPassword=my_data.loc['devPassword'].values [0]
sferaUrlLogin=my_data.loc['sferaUrlLogin'].values [0]
sferaUrl=my_data.loc['sferaUrl'].values [0]
sferaTestCaseUrl=my_data.loc['sferaTestCaseUrl'].values [0]
sferaTSectionsUrl=my_data.loc['sferaTSectionsUrl'].values [0]
sferaSprintUrl=my_data.loc['sferaSprintUrl'].values [0]
sferaUrlSearch=my_data.loc['sferaUrlSearch'].values [0]





In [3]:
session = requests.Session()
session.post(sferaUrlLogin, json={"username": devUser, "password": devPassword}, verify=False)

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sfera.inno.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [4]:
def getSuperSprintTasks(lst):
    # формируем список спринтов для запроса
    query_sprints = ''
    for sprint in lst:
        query_add = '%2C%27'
        query_end = '%27'
        if query_sprints == '':
            query_add = '%28%27'
        query_sprints += query_add + str(sprint) + query_end
    query_sprints += '%29'
    query = 'area%3D%27SKOKR%27%20and%20statusCategory%21%3D%27Done%27%20and%20type%20in%20%28%27task%27%2C%27defect%27%29%20and%20sprint%20in%20' + query_sprints + '&size=1000&page=0&attributesToReturn=checkbox%2Cnumber%2Cname%2CactualSprint%2Cpriority%2Cstatus%2Cestimation%2Cspent%2Cassignee%2Cowner%2CdueDate%2CupdateDate%2CcreateDate%2CworkGroup'
    urlQuery = sferaUrlSearch + "?query=" + query
    #urlQuery = 'https://sfera.inno.local/app/tasks/api/v0.1/entities?query=area%3D%27SKOKR%27%20and%20statusCategory%21%3D%27Done%27%20and%20type%20in%20%28%27task%27%29%20and%20sprint%20in%20%28%274246%27%2C%274247%27%2C%274248%27%2C%274249%27%2C%274250%27%2C%274251%27%29&size=1000&page=0'
    response = session.get(urlQuery, verify=False)
    if response.ok != True:
        raise Exception("Error get sprint data " + response)

    tasks = json.loads(response.text)
    return tasks

def checkTasksEstimation(tasks):
    number = []
    name = []
    estimation = []
    sprint = []
    epic = []
    type = []
    for task in tasks['content']:
        number.append(task['number'])
        name.append(task['name'])
        estimation.append(int(task['estimation']/28800))
        sprint.append(task['actualSprint']['name'])
        if 'workGroup' in task:
            type.append(task['workGroup'][0])
        else:
            type.append('Дефект')
        if 'parentNumber' in task:
            epic.append(task['parentNumber'])
        else:
            epic.append('Дефект') 

    tasks_df = pd.DataFrame({
        'number': number,
        'name': name,
        'estimation': estimation,
        'sprint': sprint,
        'epic': epic,
        'type': type
    })
    return tasks_df


tasks = getSuperSprintTasks(['4246', '4247', '4248', '4249', '4250', '4251'])
tasks_df = checkTasksEstimation(tasks)
tasks_df['type'] = tasks_df['type'].astype(str)
tasks_df['name'] = tasks_df['name'].astype(str)


c:\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sfera.inno.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
tasks_df.pivot_table(
    values='estimation',
    index='sprint',
    columns='type',
    aggfunc='sum',
    fill_value=0,
    margins=True, 
    margins_name='Total'
)

type,Архитектурная задача,Дефект,Линейная деятельность,Новая функциональность,Total
sprint,,,,,
Спринт 2024.2.1.NPP Shared Sprint,24,9,9,43,85
Спринт 2024.2.2.NPP Shared Sprint,24,7,10,44,85
Спринт 2024.2.3.NPP Shared Sprint,25,8,9,43,85
Спринт 2024.2.4.NPP Shared Sprint,25,8,7,45,85
Спринт 2024.2.5.NPP Shared Sprint,23,8,9,45,85
Спринт 2024.2.6.NPP Shared Sprint,19,8,8,50,85
Total,140,48,52,270,510


In [7]:
pivot_df = tasks_df.pivot_table(
    values='estimation',
    index=['sprint','epic'],
    columns='type',
    aggfunc='sum',
    fill_value=0,
    margins=True, 
    margins_name='Total'
)
#pivot_df.to_csv('pivot_table_results.csv', index=True)
pivot_df.to_excel('pivot_table_results.xlsx', index=True)